In [1]:
import numpy as np
import pandas as pd
import os
from itertools import permutations
from random import sample 

In [2]:
data_path = "dane/"
df = pd.read_excel(os.path.join(data_path, "dane1.xlsx"))

In [3]:
class Individual:
    def __init__(self, order):
        self.order = order
        self.value = self.__calculate_value_of_order()
        
    def __repr__(self):
        return str(self.value)
    
    def __print__(self):
        print("Order")
        print(self.order)
        print(f"Value of order: {self.value}")

    def __calculate_value_of_order(self):
        df = self.order
        if len(df) <= 0:
            return 0
        elif len(df) == 1:
            return int(df.sum(axis=1))

        obrobione = df.copy()
        obrobione["M1"] = df["M1"].cumsum() # TODO
        obrobione.iloc[1] = df.iloc[1].cumsum()

        for i in range(2, len(obrobione)):
            for j in range(2, len(obrobione.iloc[1])):
                max_val = max(int(obrobione.iloc[[i-1], [j]].values),
                                 int(obrobione.iloc[[i], [j-1]].values))
                obrobione.iloc[[i], [j]] = max_val + df.iloc[[i], [j]]
        wartosc = int(obrobione.iloc[[-1], [-1]].values)
        return wartosc
    
    def cross(self, other, cross_type = 1):
        if cross_type == 1:
            return self.__crossing_type_1(other)
        elif cross_type == 2:
            return self.__crossing_type_2(other)
    
    def __crossing_type_1(self, other):
        """
        Pierwszy sposób krzyżowania - OX crossing
            Krótki opis.
        """ 
        l = len(self.order)
        border = l//5
        left_b, right_b = np.random.randint(border, l//2), np.random.randint(l//2, l - border)

        child_1 = [None for i in range(l)]
        child_2 = [None for i in range(l)]

        for i in range(left_b, right_b):
            child_1[i] = self.order["Zadanie"].iloc[i]
            child_2[i] = other.order["Zadanie"].iloc[i]

        dropped_self = self.order.drop(self.order[self.order["Zadanie"].isin(child_2)].index)
        dropped_other = other.order.drop(other.order[other.order["Zadanie"].isin(child_1)].index)

        for i, self_task, other_task in zip(
            list(range(left_b)) + list(range(right_b, l)), 
            dropped_self["Zadanie"], 
            dropped_other["Zadanie"]
        ):
            child_1[i] = other_task
            child_2[i] = self_task

        child_1 = [x-1 for x in child_1]
        child_2 = [x-1 for x in child_2]

        child_1 = df.iloc[child_1]
        child_2 = df.iloc[child_2]
        
        child_1.index = list(range(l))
        child_2.index = list(range(l))
        
        child_1 = Individual(child_1)
        child_2 = Individual(child_2)
        
        return child_1, child_2


    def __crossing_type_2(self, other):
        """
        Operator losowy z uzupełnianiem
            Krótki opis.
        """
        l = len(self.order)
        
        child_1 = [None for i in range(l)]
        child_2 = [None for i in range(l)]

        # l//2 elementowa permutacja zbioru l
        perm_1 = sample(list(range(l)), l//2)
        perm_2 = sample(list(range(l)), l//2)
        
        perm_1
        perm_2

        for idx in perm_1:
            child_1[idx] = self.order["Zadanie"].iloc[idx]
        for idx in perm_2:
            child_2[idx] = other.order["Zadanie"].iloc[idx]

        dropped_self = self.order.drop(self.order[self.order["Zadanie"].isin(child_2)].index)
        dropped_other = other.order.drop(other.order[other.order["Zadanie"].isin(child_1)].index)

        na_idx_1 = [i for i in range(len(child_1)) if child_1[i] == None]
        na_idx_2 = [i for i in range(len(child_2)) if child_2[i] == None]

        for i_1, i_2, self_task, other_task in zip(
            na_idx_1,
            na_idx_2,
            dropped_self["Zadanie"], 
            dropped_other["Zadanie"]
        ):
            child_1[i_1] = other_task
            child_2[i_2] = self_task


        child_1 = [x-1 for x in child_1]
        child_2 = [x-1 for x in child_2]
        
        child_1 = df.iloc[child_1]
        child_2 = df.iloc[child_2]
        
        child_1.index = list(range(l))
        child_2.index = list(range(l))
        
        child_1 = Individual(child_1)
        child_2 = Individual(child_2)
        
        return child_1, child_2
        
    def mutate(self, rate = .1):
        if np.random.random() > .5:
            self.__mutate_type_1(rate)
        else:
            self.__mutate_type_2(rate)
        self.order.index = list(range(len(self.order)))
    
    def swap(self, idx):
        self.order.iloc[idx[0]], self.order.iloc[idx[1]] = self.order.iloc[idx[1]].copy(), self.order.iloc[idx[0]].copy()
    
    def __mutate_type_1(self, rate):
        l = len(self.order)
        rate = int(l*rate)
        for i in range(rate):
            idx = sample(list(range(l)), 2) # 2 losowe indexy
            self.swap(idx)
    
    def __mutate_type_2(self, rate):
        """
        Odwracanie kolejności losowo wybranego podciągu zadań.
        """
        l = len(self.order)
        rate = int(l*rate)
        point = np.random.randint(l - rate - 1)
        for i in range(point, point + rate):
            idx = (i, i+1)
            self.swap(idx)
        

In [4]:
class Population:
    def __init__(self, starting_pool, size = 20):
        self.starting_pool = starting_pool # dane wejściowe
        self.size = size # ilosc osobników w populacji
        self.list_of_individuals = self.__gen_individuals() # list osobników
        self.mating_list = [] # lista par osobników do krzyżowania
    
    def perform_selection(self, selection_type = 1):
        """
        Selekcja. Przekierowuje do odpowiedniej funkcji selekcji.
            Opis czym jest selekcja.
            Tutaj towrzymy listę rodziców.
        """
        if selection_type == 1:
            self.mating_list = self.__selection_type_1()
        elif selection_type == 2:
            self.mating_list = self.__selection_type_2()
            
    def __selection_type_1(self):
        """
        Ranking dopasowania.
        Dobiera w pary osobniki o najlepszej wartości. (Najmniejszej)
        """
        self.list_of_individuals.sort(key=lambda x : x.value)
        return list(zip(
            self.list_of_individuals[::2], 
            self.list_of_individuals[1::2]
        ))
        
    def __selection_type_2(self):
        """
        Selekcja na zasadzie turnieju
        """
        pairs = []
        
        for i in range(len(self.list_of_individuals)//2):
            fighters = sample(self.list_of_individuals, 4)

            if fighters[0].value >= fighters[1].value:
                winner_1 = fighters[0]
            else:
                winner_1 = fighters[1]

            if fighters[2].value >= fighters[3].value:
                winner_2 = fighters[2]
            else:
                winner_2 = fighters[3]

            pairs.append((winner_1, winner_2))
        return pairs
    
    def mutate(self):
        for indiv in self.list_of_individuals:
            indiv.mutate()
    
    def perform_crossing(self, crossing_type = 1):
        """
        Krzyżowanie. Przekierowuje do funckji z odpowiednim typem krzyżowania.
            Tutaj tworzymy nową populacje
        """
        children = []
        for male, female in self.mating_list:
            child_1, child_2 = male.cross(female, cross_type = crossing_type)
            children.append(child_1)
            children.append(child_2)
            
        self.mutate()
        self.__update_population(children)
        
    def __update_population(self, children):
        self.list_of_individuals = self.list_of_individuals + children
        self.list_of_individuals.sort(key=lambda x: x.value)
        self.list_of_individuals = self.list_of_individuals[:self.size]
        
    
    def show_mating_pairs(self):
        for male, female in self.mating_list:
            print(f"Male: {male.value} & Female: {female.value}")
    
    def __gen_individuals(self):
        """
        Tworzenie osobników na podstawie danych wejściowych.
        Nic nie zwraca. Wypełnia list_of_individuas.
        """
        return [Individual(self.starting_pool.sample(frac=1).reset_index(drop=True)) for x in range(self.size)]
            
    def show_individuals(self):
        """
        Zwraca reprezentację wszystkich osobników w populacji w kolejności
        od najlepszego do najgorszego.
        """
        sorted_list = sorted(self.list_of_individuals, key=lambda x: x.value)
        for indiv in sorted_list:
            print(repr(indiv))

    def get_best_individual(self):
        """
        Zwraca najlepszego osobnika w populacji.
        """
        return sorted(self.list_of_individuals, key=lambda x: x.value)[0]
        
        
        

In [5]:
population = Population(df, size=4)

for i in range(2):
    population.perform_selection()
    population.perform_crossing()

population.show_individuals()
best = population.get_best_individual()
best.order

3617
3643
3674
3681


3617

In [6]:
population = Population(df, size=4)

for i in range(2):
    population.perform_selection(selection_type=1)
    population.perform_crossing()

population.show_individuals()
best = population.get_best_individual()
best.order

3685
3704
3727
3743


3685

In [7]:
population = Population(df, size=4)

for i in range(2):
    population.perform_selection()
    population.perform_crossing(crossing_type=2)

population.show_individuals()
best = population.get_best_individual()
best.order

3584
3587
3639
3687


3584

In [8]:
population = Population(df, size=4)

for i in range(2):
    population.perform_selection(selection_type=2)
    population.perform_crossing(crossing_type=2)

population.show_individuals()
best = population.get_best_individual()
best.order

3737
3745
3750
3781


3737